## Lab 10: Patients Physical Activities prediction using Boosting


### Step 1: Understand Data


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score,accuracy_score,roc_auc_score,classification_report
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [7]:
df = pd.read_csv("Human_Activity_Data.csv")

In [8]:

df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,STANDING


In [9]:
df.shape

(151, 562)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 663.1+ KB


In [11]:
df.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-skewness()', 'fBodyBodyGyroJerkMag-kurtosis()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'Activity'],
      dtype='object', length=562)

In [12]:
type(df)

pandas.core.frame.DataFrame

In [13]:
df['Activity'].value_counts

<bound method IndexOpsMixin.value_counts of 0                STANDING
1                STANDING
2                STANDING
3                STANDING
4                STANDING
              ...        
146    WALKING_DOWNSTAIRS
147    WALKING_DOWNSTAIRS
148    WALKING_DOWNSTAIRS
149    WALKING_DOWNSTAIRS
150                   NaN
Name: Activity, Length: 151, dtype: object>

### Step 2: Build a small dataset

In [14]:
lay = df.loc[df['Activity'] == "LAYING"][:500]
sit = df.loc[df['Activity'] == "SITTING"][:500]
walk = df.loc[df['Activity'] == "WALKING"][:500]
frames = [lay, sit, walk]
df_new = pd.concat(frames)

In [15]:
df_new.shape

(98, 562)

In [16]:
df_new.to_csv("Human_Activity_sample.csv")

In [17]:
df1=pd.read_csv('Human_Activity_sample.csv')

In [18]:
df1.head()

,Unnamed: 0,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,51,0.403474,-0.015074,-0.118167,-0.914811,-0.895231,-0.891748,-0.917696,-0.924624,-0.905895,...,0.383907,-0.021764,-0.068580,-0.043105,-0.332249,0.487505,0.432479,-0.553351,-0.432876,LAYING
1,52,0.278373,-0.020561,-0.096825,-0.984883,-0.991118,-0.982112,-0.987985,-0.990362,-0.982189,...,-0.687636,-0.912512,-0.113048,0.223038,-0.393075,0.359037,0.399853,-0.550364,-0.427593,LAYING
2,53,0.276555,-0.017869,-0.107621,-0.994195,-0.996372,-0.995615,-0.994901,-0.996360,-0.996628,...,-0.519084,-0.813040,0.293145,-0.018635,-0.246980,0.453923,0.399066,-0.546725,-0.431244,LAYING
3,54,0.279575,-0.017276,-0.109481,-0.996135,-0.995812,-0.998689,-0.996393,-0.995474,-0.999006,...,-0.855139,-0.984028,0.240526,0.149804,0.393611,0.386719,0.399555,-0.546477,-0.431646,LAYING
4,55,0.276527,-0.016819,-0.107983,-0.996775,-0.997256,-0.995422,-0.997167,-0.997108,-0.995739,...,-0.852171,-0.954254,-0.347519,-0.469155,0.308855,-0.354146,0.398889,-0.546712,-0.431207,LAYING


In [19]:
df1.shape

(98, 563)

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Columns: 563 entries, Unnamed: 0 to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 431.2+ KB


In [21]:
df1.columns

Index(['Unnamed: 0', 'tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y',
       'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y',
       'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y',
       'tBodyAcc-mad()-Z',
       ...
       'fBodyBodyGyroJerkMag-skewness()', 'fBodyBodyGyroJerkMag-kurtosis()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'Activity'],
      dtype='object', length=563)

In [22]:
type(df1)

pandas.core.frame.DataFrame

In [23]:
df1["Activity"].value_counts()

WALKING    47
LAYING     27
SITTING    24
Name: Activity, dtype: int64

### Step 3: Build GradientBoostingClassifier

In [24]:
X=df1.drop('Activity',axis=1)
y=df1.Activity

In [25]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [26]:
model = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1,random_state=42)
model.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=42)

In [27]:

y_pred=model.predict(X_test)

In [28]:
accuracy_score(y_test,y_pred)

0.9666666666666667

In [29]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

      LAYING       1.00      0.90      0.95        10
     SITTING       0.89      1.00      0.94         8
     WALKING       1.00      1.00      1.00        12

    accuracy                           0.97        30
   macro avg       0.96      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



### Step4. [Find Best no. of trees and Best Learning Rate using Grid Search and Cross Validation]

In [30]:

classifier = GradientBoostingClassifier()

In [31]:
all_scores = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)

In [32]:
all_scores

array([1., 1., 1., 1., 1.])

### To find the average of all the accuracies, simple use the mean() method

In [33]:
all_scores.mean()

1.0

In [34]:
parameter = {'n_estimators': [50, 100, 200, 400], 'learning_rate': [0.1, 0.01]}

In [35]:
model1 = GridSearchCV(estimator=classifier, param_grid=parameter,cv=5, n_jobs=-1)

In [36]:
model1.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01],
                         'n_estimators': [50, 100, 200, 400]})

In [41]:
y_pred2=model1.predict(X_test)

In [42]:
accuracy_score(y_test,y_pred2)

1.0

In [43]:
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00        10
     SITTING       1.00      1.00      1.00         8
     WALKING       1.00      1.00      1.00        12

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [44]:
print(model1.best_estimator_)

GradientBoostingClassifier(n_estimators=50)


### Step5. [Build AdaBoostClassifier]

In [45]:
base = DecisionTreeClassifier()

In [46]:
model2 = AdaBoostClassifier(base_estimator=base,random_state=0)

In [47]:
param_grid = {'n_estimators': [100, 150, 200], 'learning_rate': [0.01, 0.001]}

In [48]:
model3 = GridSearchCV(model2,param_grid,cv=5,n_jobs=-1)

In [49]:
model3.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),
                                          random_state=0),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.001],
                         'n_estimators': [100, 150, 200]})

In [50]:
y_pred3=model3.predict(X_test)

In [51]:
accuracy_score(y_test,y_pred3)

1.0

In [52]:
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00        10
     SITTING       1.00      1.00      1.00         8
     WALKING       1.00      1.00      1.00        12

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [53]:
print(model3.best_estimator_)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01,
                   n_estimators=100, random_state=0)


### Step6. [Build LogisticRegressionCV classifier]

In [54]:
model4 = LogisticRegressionCV(cv=4,Cs=5,penalty='l2')

In [55]:
model4.fit(X_train,y_train)

LogisticRegressionCV(Cs=5, cv=4)

In [56]:
y_pred4=model4.predict(X_test)

In [57]:
accuracy_score(y_test,y_pred4)

1.0

In [58]:
print(classification_report(y_test,y_pred4))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00        10
     SITTING       1.00      1.00      1.00         8
     WALKING       1.00      1.00      1.00        12

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



### Step 7 [ Build VotingClassifier]

In [59]:
model5=VotingClassifier(estimators=[('lr',model4),('gbc',model1)], voting='hard')

In [60]:
model5.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegressionCV(Cs=5, cv=4)),
                             ('gbc',
                              GridSearchCV(cv=5,
                                           estimator=GradientBoostingClassifier(),
                                           n_jobs=-1,
                                           param_grid={'learning_rate': [0.1,
                                                                         0.01],
                                                       'n_estimators': [50, 100,
                                                                        200,
                                                                        400]}))])

In [63]:
y_pred5=model5.predict(X_test)

In [64]:
print(classification_report(y_test,y_pred5))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00        10
     SITTING       1.00      1.00      1.00         8
     WALKING       1.00      1.00      1.00        12

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



### Step8. [ Interpret your results]

In [65]:
print(model1.best_estimator_)

GradientBoostingClassifier(n_estimators=50)


In [66]:
print(model3.best_estimator_)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01,
                   n_estimators=100, random_state=0)


### GradientBoostingClassifier GradientBoostingClassifier(n_estimators=50)

In [67]:
classifierF = GradientBoostingClassifier(n_estimators=50)
all_scoresF = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)
parameter = {'n_estimators': [50, 100, 200, 400], 'learning_rate': [0.1, 0.01]}

In [68]:
modelGC = GridSearchCV(estimator=classifier, param_grid=parameter,cv=5, n_jobs=-1)

In [69]:
modelGC.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01],
                         'n_estimators': [50, 100, 200, 400]})

In [71]:
y_predGC=model3.predict(X_test)

In [72]:
accuracy_score(y_test,y_predGC)

1.0

In [73]:
print(classification_report(y_test,y_predGC))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00        10
     SITTING       1.00      1.00      1.00         8
     WALKING       1.00      1.00      1.00        12

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [74]:
### AdaBoostClassifier

In [75]:
#### AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01,n_estimators=100, random_state=0)

In [77]:
modelABC = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01)

In [78]:
param_grid = {'n_estimators': [100, 150, 200], 'learning_rate': [0.01, 0.001]}

In [79]:
modelGSCV = GridSearchCV(modelABC,param_grid,cv=5,n_jobs=-1)
modelGSCV.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),
                                          learning_rate=0.01),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.001],
                         'n_estimators': [100, 150, 200]})

In [80]:
y_predGSCV=model3.predict(X_test)

In [81]:
accuracy_score(y_test,y_predGSCV)

1.0

In [82]:
print(classification_report(y_test,y_predGSCV))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00        10
     SITTING       1.00      1.00      1.00         8
     WALKING       1.00      1.00      1.00        12

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

